In [74]:
import logging

import numpy as np
import pandas as pd

from datasets import binclas_datasets, regr_datasets

from config import dataset_map

In [91]:
best_params = {
    'dtc': pd.read_csv('params_dtc.csv'),
    'dtr': pd.read_csv('params_dtr.csv'),
    'rfc': pd.read_csv('params_rfc.csv'),
    'rfr': pd.read_csv('params_rfr.csv')
}

In [76]:
binclas_datasets['name'] = binclas_datasets['name'].apply(lambda x: dataset_map.get(x, x))
regr_datasets['name'] = regr_datasets['name'].apply(lambda x: dataset_map.get(x, x))

In [77]:
binclas_datasets['name_key'] = binclas_datasets.apply(lambda row: f'{row["name"]} \\cite{{{row["citation_key"]}}}', axis=1)
regr_datasets['name_key'] = regr_datasets.apply(lambda row: f'{row["name"]} \\cite{{{row["citation_key"]}}}', axis=1)

In [78]:
binclas_params = pd.merge(best_params['dtc'].rename(columns={'params': 'par_t', 'auc': 'auc_t'}), best_params['rfc'].rename(columns={'params': 'par_f', 'auc': 'auc_f'}), on=['name'])
regr_params = pd.merge(best_params['dtr'].rename(columns={'params': 'par_t', 'r2': 'r2_t'}), best_params['rfr'].rename(columns={'params': 'par_f', 'r2': 'r2_f'}), on=['name'])

In [79]:
binclas_datasets = pd.merge(binclas_datasets, binclas_params, on=['name'])
regr_datasets = pd.merge(regr_datasets, regr_params, on=['name'])

In [80]:
def clean_params(row):
    par = eval(row['par_t'])
    if 'min_samples_leaf' in par:
        val = int(np.floor(par["min_samples_leaf"] * row["n"] * 0.8))
        row['par_t'] = f'$\\alpha$: {val}'
    elif 'max_depth' in par:
        if par['max_depth'] is not None:
            row['par_t'] = f'$\\beta$: {par["max_depth"]}'
        else:
            row['par_t'] = ''

    par = eval(row['par_f'])
    if 'min_samples_leaf' in par:
        val = int(np.floor(par["min_samples_leaf"] * row["n"] * 0.8))
        row['par_f'] = f'$\\alpha$: {val}'
    elif 'max_depth' in par:
        if par['max_depth'] is not None:
            row['par_f'] = f'$\\beta$: {par["max_depth"]}'
        else:
            row['par_f'] = ''

    return row

In [81]:
binclas_datasets = binclas_datasets.apply(clean_params, axis=1)
regr_datasets = regr_datasets.apply(clean_params, axis=1)

In [82]:
binclas = binclas_datasets[['name_key', 'n', 'n_col', 'n_grid', 'n_minority', 'par_t', 'auc_t', 'par_f', 'auc_f']]\
    .rename(columns={
        'name_key': 'name',
        'n': 'N',
        'n_col': '$N_a$',
        'n_grid': '$N_l$',
        'n_minority': '$N_m$',
        'par_t': 'par$_t$',
        'auc_t': 'auc$_t$',
        'par_f': 'par$_f$',
        'auc_f': 'auc$_f$'
    })

regr = regr_datasets[['name_key', 'n', 'n_col', 'n_grid', 'par_t', 'r2_t', 'par_f', 'r2_f']]\
    .rename(columns={
        'name_key': 'name',
        'n': 'N',
        'n_col': '$N_a$',
        'n_grid': '$N_l$',
        'par_t': 'par$_t$',
        'r2_t': 'r$^2_t$',
        'par_f': 'par$_f$',
        'r2_f': 'r$^2_f$'
    })

In [83]:
binclas['name'] = binclas['name'].apply(lambda x: x.replace('_', '-'))
regr['name'] = regr['name'].apply(lambda x: x.replace('_', '-'))

In [84]:
binclas.columns = pd.MultiIndex.from_tuples([('Classification', col) for col in binclas.columns])
regr.columns = pd.MultiIndex.from_tuples([('Regression', col) for col in regr.columns])

In [85]:
binclas

Classification                                        \
                                  name    N $N_a$ $N_l$ $N_m$        par$_t$   
0             appendicitis \cite{keel}  106     7     7    21   $\alpha$: 16   
1                 haberman \cite{keel}  306     3     3    81   $\alpha$: 22   
2             new-thyroid1 \cite{keel}  215     5     5    35   $\alpha$: 15   
3                   glass0 \cite{keel}  214     9     9    70   $\alpha$: 15   
4         shuttle-6-vs-2-3 \cite{keel}  230     9     9    10    $\alpha$: 2   
5                     bupa \cite{keel}  345     6     6   145   $\alpha$: 11   
6         cleveland-0-vs-4 \cite{keel}  177    13    10    13    $\alpha$: 7   
7                   ecoli1 \cite{keel}  336     7     5    77   $\alpha$: 31   
8             poker-9-vs-7 \cite{keel}  244    10    10     8   $\alpha$: 17   
9                   monk-2 \cite{keel}  432     6     4   204    $\alpha$: 2   
10               hepatitis \cite{krnn}  155    19     6    32    $\alpha$: 6   
11    yeast-0-3-5-9-vs-7-8 \cite{keel}  506     8     6    50    $\alpha$: 9   
12            mammographic \cite{keel}  830     5     5   403   $\alpha$: 46   
13                 saheart \cite{keel}  462     9     8   160   $\alpha$: 25   
14    page-blocks-1-3-vs-4 \cite{keel}  472    10    10    28   $\alpha$: 15   
15                    pima \cite{keel}  768     8     8   268   $\alpha$: 33   
16               wisconsin \cite{keel}  683     9     9   239   $\alpha$: 13   
17             abalone9-18 \cite{keel}  731     9     7    42  $\alpha$: 116   
18  winequality-red-3-vs-5 \cite{keel}  691    11    11    10   $\alpha$: 85   

                                      
     auc$_t$       par$_f$   auc$_f$  
0   0.774544  $\alpha$: 16  0.851897  
1   0.661721  $\alpha$: 13  0.721319  
2   0.958690    $\beta$: 6  0.999802  
3   0.837056   $\beta$: 10  0.929381  
4   1.000000   $\alpha$: 1  1.000000  
5   0.695431   $\alpha$: 2  0.767310  
6   0.898073   $\alpha$: 4  0.978595  
7   0.954180   $\alpha$: 3  0.958448  
8   0.681871    $\beta$: 4  0.985988  
9   1.000000   $\alpha$: 2  1.000000  
10  0.743849    $\beta$: 2  0.871498  
11  0.736691   $\alpha$: 4  0.809820  
12  0.900469    $\beta$: 3  0.911983  
13  0.718698    $\beta$: 3  0.754571  
14  0.976071   $\beta$: 12  0.999463  
15  0.805337    $\beta$: 5  0.836252  
16  0.982664   $\alpha$: 2  0.993331  
17  0.758319   $\alpha$: 4  0.849876  
18  0.815131  $\alpha$: 17  0.869734

In [86]:
result = pd.concat([binclas, regr], axis=1)
result

Classification                                        \
                                  name    N $N_a$ $N_l$ $N_m$        par$_t$   
0             appendicitis \cite{keel}  106     7     7    21   $\alpha$: 16   
1                 haberman \cite{keel}  306     3     3    81   $\alpha$: 22   
2             new-thyroid1 \cite{keel}  215     5     5    35   $\alpha$: 15   
3                   glass0 \cite{keel}  214     9     9    70   $\alpha$: 15   
4         shuttle-6-vs-2-3 \cite{keel}  230     9     9    10    $\alpha$: 2   
5                     bupa \cite{keel}  345     6     6   145   $\alpha$: 11   
6         cleveland-0-vs-4 \cite{keel}  177    13    10    13    $\alpha$: 7   
7                   ecoli1 \cite{keel}  336     7     5    77   $\alpha$: 31   
8             poker-9-vs-7 \cite{keel}  244    10    10     8   $\alpha$: 17   
9                   monk-2 \cite{keel}  432     6     4   204    $\alpha$: 2   
10               hepatitis \cite{krnn}  155    19     6    32    $\alpha$: 6   
11    yeast-0-3-5-9-vs-7-8 \cite{keel}  506     8     6    50    $\alpha$: 9   
12            mammographic \cite{keel}  830     5     5   403   $\alpha$: 46   
13                 saheart \cite{keel}  462     9     8   160   $\alpha$: 25   
14    page-blocks-1-3-vs-4 \cite{keel}  472    10    10    28   $\alpha$: 15   
15                    pima \cite{keel}  768     8     8   268   $\alpha$: 33   
16               wisconsin \cite{keel}  683     9     9   239   $\alpha$: 13   
17             abalone9-18 \cite{keel}  731     9     7    42  $\alpha$: 116   
18  winequality-red-3-vs-5 \cite{keel}  691    11    11    10   $\alpha$: 85   

                                                            Regression        \
     auc$_t$       par$_f$   auc$_f$                              name     N   
0   0.774544  $\alpha$: 16  0.851897              diabetes \cite{keel}    43   
1   0.661721  $\alpha$: 13  0.721319                 o-ring \cite{uci}    23   
2   0.958690    $\beta$: 6  0.999802                wsn-ale \cite{uci}   107   
3   0.837056   $\beta$: 10  0.929381           daily-demand \cite{uci}    60   
4   1.000000   $\alpha$: 1  1.000000            slump-test \cite{krnn}   103   
5   0.695431   $\alpha$: 2  0.767310                  servo \cite{uci}   167   
6   0.898073   $\alpha$: 4  0.978595   yacht-hydrodynamics \cite{krnn}   307   
7   0.954180   $\alpha$: 3  0.958448              autoMPG6 \cite{keel}   392   
8   0.681871    $\beta$: 4  0.985988     excitation-current \cite{uci}   557   
9   1.000000   $\alpha$: 2  1.000000  real-estate-valuation \cite{uci}   414   
10  0.743849    $\beta$: 2  0.871498               wankara \cite{keel}   321   
11  0.736691   $\alpha$: 4  0.809820               plastic \cite{keel}  1650   
12  0.900469    $\beta$: 3  0.911983                 laser \cite{keel}   993   
13  0.718698    $\beta$: 3  0.754571  qsar-aquatic-toxicity \cite{uci}   546   
14  0.976071   $\beta$: 12  0.999463              baseball \cite{keel}   337   
15  0.805337    $\beta$: 5  0.836252   maternal-health-risk \cite{uci}  1013   
16  0.982664   $\alpha$: 2  0.993331       cpu-performance \cite{krnn}   209   
17  0.758319   $\alpha$: 4  0.849876               airfoil \cite{krnn}  1503   
18  0.815131  $\alpha$: 17  0.869734       medical-cost \cite{mlwithr}  1338   

                                                               
   $N_a$ $N_l$       par$_t$   r$^2_t$      par$_f$   r$^2_f$  
0      2     2   $\alpha$: 6 -0.375384   $\beta$: 2 -0.083344  
1      6     4   $\alpha$: 1  0.126992   $\beta$: 2  0.150356  
2      5     4   $\alpha$: 5  0.433964  $\alpha$: 2  0.560735  
3     12     7   $\alpha$: 1  0.697130   $\beta$: 7  0.828057  
4      9     9   $\alpha$: 2  0.623463   $\beta$: 8  0.768588  
5     10     2   $\alpha$: 4  0.685812  $\alpha$: 2  0.720899  
6      6     6   $\alpha$: 1  0.993250  $\beta$: 11  0.994930  
7      5     5  $\alpha$: 10  0.822720  $\beta$: 12  0.872429  
8      4     4   $\bet

In [87]:
latex = result.to_latex(
    index=False,
    formatters={col: lambda x: f'{x:.3f}' for col in result.columns if '^2' in col[-1] or 'auc' in col[-1]},
    multicolumn_format='c'
    )

In [88]:
tabular_string = latex[len('\\begin{tabular}{'): len('\\begin{tabular}{') + len(result.columns)]
tabular_string


'lrrrrlrlrlrrrlrlr'

In [89]:
updated = '@{\hspace{6pt}}'.join(tabular_string)
latex = latex.replace(tabular_string, updated)

In [90]:
with open('tab_datasets.tex', 'wt') as file:
    file.write(latex)